### Import modules

In [ ]:
# Extinction Strain Rate with Sensitivity Alalysis
# Run with Cantera version 2.3.0 or higher
# Written by Alon Grinberg Dana, inspired by:
# (1) twin_premixed_flame_axisymmetric.ipynb in Cantera, and
# (2) http://www.cantera.org/docs/sphinx/html/cython/examples/onedim_diffusion_flame_extinction.html
#     cite: doi:10.1155/2014/484372)

from __future__ import print_function
from __future__ import division
import cantera as ct
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import time
%matplotlib notebook

print("Running Cantera Version: " + str(ct.__version__))

### User viriables: reactant conditions, kinetic mechanism, domain length

In [ ]:
# Setting the inlet T and P
To = 300     # K
Po = 101325  # Pa

# Setting the composition
composition = 'CH4:0.75, O2:2, N2: 7.52'

# Define the gas-mixutre and kinetics
gas = ct.Solution('gri30.cti')

# Define a domain half-width of x m, i.e., the whole domain is 2x m wide
width = 0.007 # m

gas.TPX = To, Po, composition

# Set the reactants inlet velocity which determines the strain-rate
axial_velocity = 200 # in cm/s

# Compute the mass flux, as required by the Flame object
massFlux = gas.density * axial_velocity * 0.01 # units kg/m2/s

### Define functions

In [ ]:
# Differentiation function for data that has variable grid spacing, used to compute normal strain-rate
def derivative(x, y):
    dydx = np.zeros(y.shape, y.dtype.type)

    dx = np.diff(x)
    dy = np.diff(y)
    dydx[0:-1] = dy/dx

    dydx[-1] = (y[-1] - y[-2])/(x[-1] - x[-2])

    return dydx

def computeStrainRates(oppFlame):
    # Compute the derivative of axial velocity to obtain normal strain rate
    strainRates = derivative(oppFlame.grid, oppFlame.u)

    # Obtain the Characteristic Strain Rate (K)
    n = 0
    while strainRates[n] > strainRates[n+1]:
        n +=1
    strainRatePoint = n
    K = abs(strainRates[n])
    
    return strainRates, strainRatePoint, K

def computeConsumptionSpeed(oppFlame):

    Tb = max(oppFlame.T)
    Tu = min(oppFlame.T)
    rho_u = max(oppFlame.density)

    integrand = oppFlame.heat_release_rate/oppFlame.cp

    I = np.trapz(integrand, oppFlame.grid)
    Sc = I/(Tb - Tu)/rho_u

    return Sc

# This function is called to run the solver
def solveOpposedFlame(oppFlame, massFlux=0.12, loglevel=0,
                      ratio=2, slope=0.3, curve=0.3, prune=0.05):
    """
    Execute this function to run the Oppposed Flow Simulation. This function
    takes a CounterFlowTwinPremixedFlame object as the first argument
    """

    oppFlame.reactants.mdot = massFlux
    oppFlame.set_refine_criteria(ratio=ratio, slope=slope, curve=curve, prune=prune)

    oppFlame.show_solution()
    oppFlame.solve(loglevel, auto=True)

    # Compute the strain rate
    strainRates, strainRatePoint, K = computeStrainRates(oppFlame)

    return np.max(oppFlame.T), K, strainRatePoint

### Run the Solver

In [ ]:
# Create the flame object
oppFlame = ct.CounterflowTwinPremixedFlame(gas, width=width)

# Uncomment the following line to use a Multi-component transport formulation (default is 'mixture-averaged')
#oppFlame.transport_model = 'Multi'


# The solver returns the peak temperature, strain rate and
# the point which we ascribe to the characteristic strain rate.

(T, K, strainRatePoint) = solveOpposedFlame(oppFlame, massFlux, loglevel=0)

# You can plot/see all state space variables by calling oppFlame.foo where foo
# is T, Y[i], etc. The spatial variable (distance in meters) is in oppFlame.grid
# Thus to plot temperature vs. distance, use oppFlame.grid and oppFlame.T

#This is to save output
oppFlame.write_csv("premixed_twin_flame.csv", quiet=False)

Sc = computeConsumptionSpeed(oppFlame)

print("Peak temperature: {0:.1f} K".format(T))
print("Strain Rate: {0:.1f} 1/s".format(K))
print("Consumption Speed: {0:.2f} cm/s".format(Sc*100))

### Plot figures

In [ ]:
# define plotting preference

plt.rcParams['figure.autolayout'] = True

plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 10
plt.rcParams['figure.facecolor'] = 'white'
plt.rcParams['figure.figsize'] = (8,6)